In [1]:
import tensorflow as tf
import pathlib
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from custom_fn import specificity,recall

2023-11-23 23:38:30.359944: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2023-11-23 23:38:30.384540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-23 23:38:30.384565: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-23 23:38:30.384580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-23 23:38:30.389517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = pathlib.Path('./Datasets/lung/').with_suffix('')

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  label_mode="binary",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

Found 977 files belonging to 2 classes.
Using 489 files for training.


2023-11-23 23:38:31.281476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 23:38:31.284374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-23 23:38:31.284485: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  label_mode="binary",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

Found 977 files belonging to 2 classes.
Using 390 files for validation.


In [5]:
model=Sequential()
model.add(Conv2D(filters= 8,kernel_size=(4,4),input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters= 16,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters= 32,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(units=16,activation='relu'))
model.add(Dense(units=8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 8)       392       
                                                                 
 conv2d_1 (Conv2D)           (None, 510, 510, 16)      1168      
                                                                 
 max_pooling2d (MaxPooling2  (None, 255, 255, 16)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 253, 253, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 124, 124, 64)      1

In [6]:
model.compile(
  optimizer='adam',
  loss=keras.losses.BinaryCrossentropy(),
  metrics=['accuracy',specificity,recall])

In [7]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=6,verbose=1)
mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",save_best_only=True)
cd=[es,mc]

In [8]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=30,
  use_multiprocessing=True,
  callbacks=cd
)

Epoch 1/30


2023-11-23 23:38:32.807971: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-23 23:38:33.392653: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.70GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-23 23:38:33.417734: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.25GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-23 23:38:34.027746: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

16/16 [==============================] - ETA: 0s - loss: 24.8144 - accuracy: 0.5112 - specificity: 0.6046 - recall: 0.4023
Epoch 1: val_accuracy improved from -inf to 0.62051, saving model to ./bestmodel.h5
16/16 [==============================] - 12s 397ms/step - loss: 24.8144 - accuracy: 0.5112 - specificity: 0.6046 - recall: 0.4023 - val_loss: 0.6146 - val_accuracy: 0.6205 - val_specificity: 1.0000 - val_recall: 0.0433
Epoch 2/30


/home/star/Projects/Machine Learning/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


16/16 [==============================] - ETA: 0s - loss: 0.3845 - accuracy: 0.8241 - specificity: 0.8995 - recall: 0.3746
Epoch 2: val_accuracy improved from 0.62051 to 0.80769, saving model to ./bestmodel.h5
16/16 [==============================] - 4s 263ms/step - loss: 0.3845 - accuracy: 0.8241 - specificity: 0.8995 - recall: 0.3746 - val_loss: 0.3571 - val_accuracy: 0.8077 - val_specificity: 0.6487 - val_recall: 0.6210
Epoch 3/30
16/16 [==============================] - ETA: 0s - loss: 0.1340 - accuracy: 0.9448 - specificity: 0.9532 - recall: 0.4251
Epoch 3: val_accuracy improved from 0.80769 to 0.96923, saving model to ./bestmodel.h5
16/16 [==============================] - 4s 263ms/step - loss: 0.1340 - accuracy: 0.9448 - specificity: 0.9532 - recall: 0.4251 - val_loss: 0.1032 - val_accuracy: 0.9692 - val_specificity: 0.9945 - val_recall: 0.3958
Epoch 4/30
16/16 [==============================] - ETA: 0s - loss: 0.0405 - accuracy: 0.9877 - specificity: 0.9901 - recall: 0.4143
Epoc